In [78]:
import time
import base64
import random
import secrets
import hashlib
import string
import numpy as np
from collections import Counter
from sympy import nextprime
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import hashes


In [80]:
class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(text):
    freq = Counter(text)
    nodes = [Node(char, freq) for char, freq in freq.items()]
    while len(nodes) > 1:
        nodes.sort()
        left = nodes.pop(0)
        right = nodes.pop(0)
        merged = Node(None, left.freq + right.freq)
        merged.left = left
        merged.right = right
        nodes.append(merged)
    return nodes[0]

In [82]:
def generate_huffman_codes(node, prefix="", code_dict=None):
    if code_dict is None:
        code_dict = {}
    
    if node:
        if node.char is not None: 
            code_dict[node.char] = prefix
        generate_huffman_codes(node.left, prefix + "0", code_dict)
        generate_huffman_codes(node.right, prefix + "1", code_dict)
    
    return code_dict

def huffman_encode(text, huff_codes):
    return "".join(huff_codes[char] for char in text)

In [84]:
import base64

def xor_with_key(binary_text, key_bytes):
    # If key is bytes, ensure it's base64 encoded string first
    key = base64.b64encode(key_bytes).decode()  # Convert bytes → base64 → string
    
    # Add base64 padding if needed
    missing_padding = len(key) % 4
    if missing_padding:
        key += '=' * (4 - missing_padding)

    # Decode the base64 key back to bytes
    key_bytes = base64.b64decode(key)

    # Convert to binary
    key_bin = ''.join(format(byte, '08b') for byte in key_bytes)

    # XOR the binary_text with the key
    xor_result = ''.join(
        str(int(binary_text[i]) ^ int(key_bin[i % len(key_bin)]))
        for i in range(len(binary_text))
    )

    return xor_result


In [86]:
#DNA Encoding
def binary_to_dna(binary_str):
    dna_mapping = {"00": "A", "01": "T", "10": "G", "11": "C"}
    padded_binary = binary_str + "0" * ((4 - len(binary_str) % 4) % 4)
    dna_encoded = "".join(dna_mapping[padded_binary[i:i+2]] for i in range(0, len(padded_binary), 2))
    return dna_encoded, len(padded_binary) - len(binary_str)

def dna_to_rna(dna_str):
    return dna_str.replace("T", "U")

In [88]:
def adaptive_permutation(text, key):
    # Ensure key is a string
    if isinstance(key, (bytes, bytearray)):
        key = key.decode(errors='ignore')

    block_size = 4  # You can change the block size
    key_sum = sum(ord(c) for c in key)

    # Generate a fixed permutation pattern using the key
    base_pattern = list(range(block_size))
    for i in range(key_sum % 10):  # A deterministic "shuffle" using left rotation
        base_pattern = base_pattern[1:] + base_pattern[:1]

    result = []
    for i in range(0, len(text), block_size):
        block = text[i:i + block_size]
        permuted = ''.join(block[j] for j in base_pattern if j < len(block))
        result.append(permuted)

    return ''.join(result)


In [90]:
from Bio.Seq import Seq
from Bio.Data import CodonTable


standard_codon_table = CodonTable.unambiguous_rna_by_name["Standard"]

def rna_to_protein(rna_sequence):
    padding = (3 - len(rna_sequence) % 3) % 3
    padded_rna = rna_sequence + "A" * padding  # Use 'A' as neutral padding
    
    print(f" RNA Before Protein Translation: {padded_rna} (Padding: {padding})")

    codons = [padded_rna[i:i+3] for i in range(0, len(padded_rna), 3)]
    
    # Translate RNA to Protein
    protein_seq = str(Seq(padded_rna).translate(to_stop=False))

    print(f"Translated Protein: {protein_seq}")
    print(f" Codon Map: {codons}")
    
    return protein_seq, padding, codons

def protein_to_rna(protein_seq, padding, codon_map):
    print(f" Protein Before RNA Translation: {protein_seq}")

    rna_sequence = "".join(codon_map[i] for i in range(len(protein_seq)))

    if padding:
        rna_sequence = rna_sequence[:-padding]

    print(f" RNA After Reverse Translation: {rna_sequence}")
    return rna_sequence


In [92]:
key="ºî"
def encrypt(plain_text, key):
    print("Encryption")

    #Huffman Encoding
    huff_tree = build_huffman_tree(plain_text)
    huff_codes = generate_huffman_codes(huff_tree)
    huffman_encoded = huffman_encode(plain_text, huff_codes)
    print(f"Huffman Encoded Binary: {huffman_encoded}")
    
    #XOR Encryption
    xor_result = xor_with_key(huffman_encoded, key)
    print(f"After XOR Encryption: {xor_result}")

    #DNA Encoding
    dna_encoded, dna_padding = binary_to_dna(xor_result)
    print(f"After DNA Encoding: {dna_encoded}")

    #RNA Conversion
    rna_encoded = dna_to_rna(dna_encoded)
    print(f"After RNA Encoding: {rna_encoded}")

    #Adaptive Permutation
    permuted_text = adaptive_permutation(rna_encoded, key)
    print(f"After Permutation: {permuted_text}")

    # Step 6: RNA to Protein Conversion
    protein_seq, protein_padding ,codon_map= rna_to_protein(permuted_text)
    print(f"Protein Sequence: {protein_seq} ")
    
    return protein_seq, huff_codes, dna_padding, protein_padding,codon_map


In [94]:
def rna_to_dna(rna_str):
    return rna_str.replace("U", "T")
def dna_to_binary(dna_str):
    dna_mapping_reverse = {"A": "00", "T": "01", "G": "10", "C": "11"}
    binary_str = "".join(dna_mapping_reverse[base] for base in dna_str)
    return binary_str

In [96]:
def reverse_adaptive_permutation(permuted_text, key):
    if isinstance(key, (bytes, bytearray)):
        key = key.decode(errors='ignore')

    block_size = 4
    key_sum = sum(ord(c) for c in key)

    base_pattern = list(range(block_size))
    for i in range(key_sum % 10):
        base_pattern = base_pattern[1:] + base_pattern[:1]

    inverse_pattern = [0] * len(base_pattern)
    for i, pos in enumerate(base_pattern):
        inverse_pattern[pos] = i

    result = []
    for i in range(0, len(permuted_text), block_size):
        block = permuted_text[i:i + block_size]
        temp = [''] * len(block)
        for j in range(len(block)):
            if inverse_pattern[j] < len(block):
                temp[inverse_pattern[j]] = block[j]
        result.append(''.join(temp))

    return ''.join(result)


In [98]:
def decrypt(permuted_text, key, huffman_codes, dna_padding, protein_padding):
    print("Starting Decryption...")

    rna_text = protein_to_rna(permuted_text, protein_padding, codon_map)
    print(f"After Protein to RNA Conversion: {rna_text}")

    reversed_text = reverse_adaptive_permutation(rna_text, key)
    print(f"After Reverse Permutation: {reversed_text}")

    dna_text = rna_to_dna(reversed_text)
    print(f"After RNA to DNA Conversion: {dna_text}")

    binary_text = dna_to_binary(dna_text)
    print(f"After DNA to Binary Conversion: {binary_text}")

    # Step 5: Remove Padding
    original_length = len(binary_text)
    if dna_padding:
        binary_text = binary_text[:-(dna_padding)]  
    print(f" Binary Length Before Padding Removal: {original_length}")
    print(f" Binary Length After Padding Removal: {len(binary_text)}")
    print(f" After Removing Padding: {binary_text}")

    # Step 6: XOR Decryption
    original_xor = xor_with_key(binary_text, key)
    print(f"After XOR Decryption: {original_xor}")

    # Step 7: Huffman Decoding
    inverted_huffman_codes = {v: k for k, v in huffman_codes.items()}
    decoded_text = ""
    buffer = ""
    for bit in original_xor:
        buffer += bit
        if buffer in inverted_huffman_codes:
            decoded_text += inverted_huffman_codes[buffer]
            print(f"Decoded Character: {inverted_huffman_codes[buffer]} from Buffer: {buffer}")
            buffer = ""

    if buffer:
        print(f" Huffman decoding failed! Buffer left: {buffer}")
        print(f"Incomplete Huffman Decoding: {decoded_text}")

    print(f"Decrypted Text: {decoded_text}")
    print(f"Decryption Time: {time.time() - start_time:.6f} seconds\n")

    return decoded_text


In [100]:
import hashlib
from cryptography.hazmat.primitives.asymmetric import dh

parameters = dh.generate_parameters(generator=2, key_size=512)

def generate_dh_keys(parameters):
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

def derive_16bit_shared_secret(private_key, peer_public_key):
    shared_secret = private_key.exchange(peer_public_key)
    digest = hashlib.sha256(shared_secret).digest()  # SHA-256 output is 32 bytes
    return digest[:2]  # First 2 bytes = 16 bits

private_key_A, public_key_A = generate_dh_keys(parameters)
private_key_B, public_key_B = generate_dh_keys(parameters)

shared_secret_A = derive_16bit_shared_secret(private_key_A, public_key_B)
shared_secret_B = derive_16bit_shared_secret(private_key_B, public_key_A)

assert shared_secret_A == shared_secret_B

final_key= shared_secret_A

print("16-bit DH Key (raw bytes):", final_key)


16-bit DH Key (raw bytes): b'\x87\xba'


In [102]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()
def write_to_file(file_path, data):
    with open(file_path, 'w') as file:
        file.write(data)
input_path = "input.txt"
output_path = "output.txt"
plain_text = read_file(input_path)

start_time = time.time()
encrypted, huffman_codes, dna_padding, protein_padding, codon_map = encrypt(plain_text, final_key)
end_time = time.time()

print(f"Encryption Time: {end_time - start_time:.6f} seconds")

write_to_file(output_path, encrypted)
print(f"Encrypted text saved to: {output_path}")


Encryption
Huffman Encoded Binary: 11101111101011000000111001010011
After XOR Encryption: 01101000000101101000100111101001
After DNA Encoding: TGGAATTGGAGTCGGT
After RNA Encoding: UGGAAUUGGAGUCGGU
After Permutation: UGGAAUUGGAGUCGGU
 RNA Before Protein Translation: UGGAAUUGGAGUCGGUAA (Padding: 2)
Translated Protein: WNWSR*
 Codon Map: ['UGG', 'AAU', 'UGG', 'AGU', 'CGG', 'UAA']
Protein Sequence: WNWSR* 
Encryption Time: 0.001547 seconds
Encrypted text saved to: output.txt


In [104]:
def write_to_file(file_path, data):
    with open(file_path, 'w') as file:
        file.write(data)
decrypted_output_path = "output.txt"
start_time = time.time()
decrypted = decrypt(encrypted, final_key, huffman_codes, dna_padding, protein_padding)
end_time = time.time()

print(f"Decryption Time: {end_time - start_time:.6f} seconds")
write_to_file(decrypted_output_path, decrypted)
print(f"Decrypted text saved to: {decrypted_output_path}")


Starting Decryption...
 Protein Before RNA Translation: WNWSR*
 RNA After Reverse Translation: UGGAAUUGGAGUCGGU
After Protein to RNA Conversion: UGGAAUUGGAGUCGGU
After Reverse Permutation: UGGAAUUGGAGUCGGU
After RNA to DNA Conversion: TGGAATTGGAGTCGGT
After DNA to Binary Conversion: 01101000000101101000100111101001
 Binary Length Before Padding Removal: 32
 Binary Length After Padding Removal: 32
 After Removing Padding: 01101000000101101000100111101001
After XOR Decryption: 11101111101011000000111001010011
Decoded Character: H from Buffer: 1110
Decoded Character: e from Buffer: 1111
Decoded Character: l from Buffer: 10
Decoded Character: l from Buffer: 10
Decoded Character: o from Buffer: 110
Decoded Character:   from Buffer: 000
Decoded Character: W from Buffer: 001
Decoded Character: o from Buffer: 110
Decoded Character: r from Buffer: 010
Decoded Character: l from Buffer: 10
Decoded Character: d from Buffer: 011
Decrypted Text: Hello World
Decryption Time: 0.000652 seconds

Decrypt